In [ ]:
import sys
sys.path.append("../src/")

In [ ]:
import s3fs

fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},key ='MDZIFBFIOR8U3VRAVPQU', secret = 'gRNTize+y228JCco09Kb3V6X8ZYTkRMBBEtAS01N', token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJNRFpJRkJGSU9SOFUzVlJBVlBRVSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNjk3MTI4MjkxLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6InRvbS5zZWltYW5kaUBpbnNlZS5mciIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJleHAiOjE2OTcyNzE4MjIsImZhbWlseV9uYW1lIjoiU2VpbWFuZGkiLCJnaXZlbl9uYW1lIjoiVG9tIiwiZ3JvdXBzIjpbImFwZSIsImNvZGlmaWNhdGlvbi1lZHQiLCJjb2RpZmljYXRpb24tcGNzIiwiY29ldXIydmlsbGUiLCJkZWR1cC1vamEiLCJlbnF1ZXRlLWFzc29jaWF0aW9ucyIsImV4dHJhY3Rpb24tdGFibGVhdXgiLCJmb3JtYXRpb24iLCJmb3JtaWFibGUiLCJmdW5hdGhvbiIsImhhY2thdGhvbi1udHRzLTIwMjMiLCJoYWNrYXRob24tdW4tMjAyMiIsIm1vYml0aWMiLCJtdXR1YWxpc2F0aW9uLWNvZGlmaWNhdGlvbiIsInByZWRpY2F0Iiwic2lnbmVzLWRlLXZpZSIsInNpdGVzc3BsYWIiLCJzbHVtcy1kZXRlY3Rpb24iLCJzb2NyYXRleHQiLCJzc3BsYWIiLCJ0ZWxtb2IiXSwiaWF0IjoxNjk3MTI4MjkyLCJpc3MiOiJodHRwczovL2F1dGgubGFiLnNzcGNsb3VkLmZyL2F1dGgvcmVhbG1zL3NzcGNsb3VkIiwianRpIjoiY2FlMGRjNzItNmRjYy00NzU5LThhZjItODcxNTEwMTI0NmQxIiwibG9jYWxlIjoiZW4iLCJuYW1lIjoiVG9tIFNlaW1hbmRpIiwicG9saWN5Ijoic3Rzb25seSIsInByZWZlcnJlZF91c2VybmFtZSI6InRzZWltYW5kaSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBncm91cHMgZW1haWwiLCJzZXNzaW9uX3N0YXRlIjoiZjk3NjczMmQtYjdmZi00ZDU5LTk3ZGQtMzc5YjYzNDM3ZTJjIiwic2lkIjoiZjk3NjczMmQtYjdmZi00ZDU5LTk3ZGQtMzc5YjYzNDM3ZTJjIiwic3ViIjoiZWYxYmUxMTItNzAzMy00ODg0LTg4NzEtMjgwOTU2OWIwNzdiIiwidHlwIjoiQmVhcmVyIn0.RaVEiclZiAWzGIwPvQJFBljQYVjHMUT19uV5JgBEOLEAEbx98Ujr1r6aD9YTzF0talJdh31s478grx0dptXxCQ')

In [ ]:
fs.ls("projet-slums-detection/Donnees/ADMINEXPRESS/guadeloupe/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGAF09UTM20_GLP")

In [ ]:
import geopandas as gpd

region_border = gpd.read_file("s3://projet-slums-detection/Donnees/ADMINEXPRESS/guadeloupe/1_DONNEES_LIVRAISON_2023-09-21/ADE_3-2_SHP_RGAF09UTM20_GLP/REGION.shp")

In [ ]:
region_border.geometry.plot()

In [ ]:
from pathlib import Path
import re

def dep_from_train_path(path: str):
    """
    Get and return dep from train image path.

    Args:
        path (str): Path of train image.
    Return:
        str: Département.
    """
    file_name = Path(path).name
    pattern = r"^(.*?)-"

    # Use re.match to find the match at the beginning of the string
    match = re.match(pattern, file_name)

    if match:
        dep = match.group(1)
        if dep not in [
            "mayotte",
            "guadeloupe",
            "martinique",
            "guyane"
        ]:
            raise ValueError(f"Département {dep} not valid.")
    else:
        raise ValueError("Département not found in path.")
    return dep

In [ ]:
# Get a few satellite images
paths = [
    "/home/onyxia/work/train_data2-detection-PLEIADES-BDTOPO-detection-dummy-2022/images/mayotte-ORT_2020052526670967_0522_8592_U38S_8Bits_0000.jp2",
    "/home/onyxia/work/train_data2-detection-PLEIADES-BDTOPO-detection-dummy-2022/images/mayotte-ORT_2020052526670967_0522_8592_U38S_8Bits_0001.jp2",
    "/home/onyxia/work/train_data2-detection-PLEIADES-BDTOPO-detection-dummy-2022/images/mayotte-ORT_2020052526670967_0522_8592_U38S_8Bits_0002.jp2",
    "/home/onyxia/work/train_data2-detection-PLEIADES-BDTOPO-detection-dummy-2022/images/mayotte-ORT_2020052526670967_0522_8592_U38S_8Bits_0003.jp2",
]
dep_from_train_path(paths[0])

In [ ]:

img = SatelliteImage.from_raster(file_path=paths[0], dep=None, date=None, n_bands=3).array